In [1]:
import os
import pandas as pd
pd.options.display.max_colwidth = 3000
from nltk.corpus import stopwords   
import warnings
warnings.filterwarnings("ignore")
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import matplotlib
import sklearn.feature_extraction.text as text
import gensim
import smart_open
import pprint as pp
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import numpy as np
from collections import defaultdict
from gensim import corpora
from gensim import models
import pprint as pprint
from nltk.corpus import stopwords
from matplotlib import pyplot as plt


def texts_to_df(path,rstr):

    file_names = os.listdir(path)
    # Create Dictionary for File Name and Text
    file_name_and_text = {}
    for file in file_names:
        with open(path+ '/' + file, "r", encoding='utf-8') as target_file:
            next(target_file)
            file_name_and_text[file] = target_file.read()

    file_data = (pd.DataFrame.from_dict(file_name_and_text, orient='index').reset_index().rename(index=str, columns={'index': 'file_name', 0: 'content'}))

    lines_list = []
    for file in file_names:
        with open(path+'/' + file, "r", encoding='utf-8') as target_line:

            file_name = target_line.readline()
            file_name = file_name.rstrip('\n')
            file_name = file_name.rstrip(rstr)
        lines_list.append(file_name)
    lines = pd.DataFrame(lines_list)
    lines.columns=['headline']
    lines=lines.reset_index(drop=True,inplace=False)
    file_data = file_data.drop('file_name',axis=1)
    file_data=file_data.reset_index(drop=True,inplace=False)
    final_df = pd.concat([lines,file_data],axis=1)
    final_df=final_df.drop_duplicates(subset='headline',keep='first')
    final_dff=final_df["headline"].str.rstrip("-")
    final_dff=pd.DataFrame(final_dff)
    final_dff.columns=['headlines']
    final_df=pd.concat([final_dff,final_df],axis=1)
    final_df = final_df.drop('headline',axis=1)
    final_df['content'] = final_df['content'].apply(lambda x: ' '.join(x.split(' ')[75:]))
    final_df['content'] = final_df['content'].apply(lambda x: ' '.join(x.split(' ')[:400]))
    final_df.reset_index(drop=True,inplace=True)
    return final_df


data_nyt=texts_to_df(path='nyt tweets',rstr=' The New York Times')
# data_cnn = texts_to_df(path='cnn_tweets',rstr='-')
data_guardian = texts_to_df(path='guardian tweets',rstr=' | ')
data_cbs = texts_to_df(path='cbs tweets',rstr=' - CBS News')
data_abc = texts_to_df(path='abc_news',rstr='- ABC News')


    


2020-05-21 17:15:35,337 : INFO : 'pattern' package not found; tag filters are not available for English


In [2]:
data_nyt

,headlines,content
0,How Bad Is Unemployment? ‘Literally Off the Charts’,"the worst devastation since the Great Depression.\nThe Labor Department’s monthly report on Friday provided the clearest picture yet of the breadth and depth of the economic damage — and how swiftly it spread — as the coronavirus pandemic swept the country.\nJob losses have encompassed the entire economy, affecting every major industry. Areas like leisure and hospitality had the biggest losses in April, but even health care shed more than a million jobs. Low-wage workers, including many women and members of racial and ethnic minorities, have been hit especially hard.\n“It’s literally off the charts,” said Michelle Meyer, head of U.S. economics at Bank of America. “What would typically take months or quarters to play out in a recession happened in a matter of weeks this time.”\nUnemployment rate\nSource: Department of Labor\nFrom almost any vantage point, it was a bleak report. The share of the adult population with a job, at 51.3 percent, was the lowest on record. Nearly 11 million people reported working part time because they couldn’t find full-time work, up from about four million before the pandemic.\nIf anything, the numbers probably understate the economic distress.\nMillions more Americans have filed unemployment claims since the data was collected in mid-April. What’s more, because of issues with the way workers are classified, the Labor Department said the actual unemployment rate last month might have been closer to 20 percent.\nIt remains possible that the recovery, too, will be swift, and that as the pandemic retreats, businesses that were fundamentally healthy before the virus will reopen, rehire and return more or less to normal. The one bright spot in Friday’s report was that nearly 80 percent of the unemployed said they had been temporarily laid off and expected to return to their jobs in the coming months.\nShare of unemployed on temporary layoffs\nSource: Department of Labor\nPresident Trump endorsed this view in an interview Friday morning on Fox News. “Those jobs will all be back, and they’ll be back very soon,” Mr. Trump said, “and next year we’re going to have a phenomenal year.”\nBut Diane Swonk, chief economist at Grant Thornton, said that such optimism was misplaced, and that many of the jobs could not be recovered.\n“This is going to be a hard reality,” Ms. Swonk said. “These furloughs are permanent, not temporary.”\nMany businesses have indicated that employees can work from home throughout the summer, hurting sales at downtown restaurants. Meetings and conferences have been put off"
1,The Disastrous Employment Numbers Show Almost Every Job Is at Risk,"8, 2020Updated May 9, 2020, 12:19 p.m. ETListen to This ArticleAudio Recording by AudmTo hear more audio stories from publishers like The New York Times, download Audm for iPhone or Android.The jobs numbers were the catastrophe everybody was expecting.April 2020 — more technically, the period between the second week of March and the second week of April — was the worst month for American workers at least since the Great Depression and possibly in the history of the nation.That isn’t really a surprise, but one aspect of the latest employment report does help crystallize the nature of what the United States is grappling with. In a set of tables in the final pages of the jobs numbers, the Bureau of Labor Statistics reports the number of jobs gained or lost in each industry, broken down in a fairly fine-grained way.Across dozens and dozens of industries, only one added a meaningful number of jobs in April: general merchandise stores, including warehouse clubs and supercenters. They increased their payrolls by 93,400 positions.That makes sense given Americans need to buy groceries and other at-home staples, and Walmart has said publicly that it is hiring on a large scale to meet demand.There were a few other sectors with very narrowly positive numbers, including manufacturers of compu

In [3]:
all_news=pd.concat([data_nyt,data_guardian,data_cbs,data_abc])
all_news=all_news.drop_duplicates(keep='first',subset='headlines')
all_news.reset_index(drop=True,inplace=True)
all_news = all_news.sample(frac=1)
all_news.reset_index(inplace=True,drop=True)
# all_news_for_training=all_news[:18000]
# all_news_for_testing = all_news[18000:]

In [4]:
all_news

,headlines,content
0,"As Pandemic Rages, Sri Lanka’s President Pardons a War Criminal","time as defense secretary, when he oversaw the last bloody years of the country’s civil war.Credit...Dinuka Liyanawatte/ReutersBy Maria Abi-Habib and Dharisha BastiansMarch 28, 2020Sri Lanka’s president has pardoned a soldier who was sentenced to death for killing eight civilians during the country’s civil war, leading to accusations that the government was taking advantage of the chaos from the coronavirus pandemic to free a wartime ally accused of atrocities.The pardon reverses one of the very few convictions from the 26-year civil war, during which dozens of militants and military officers were accused of war crimes. The pardoned soldier, former Staff Sgt. Sunil Ratnayake, was sentenced in 2015 for blindfolding eight civilians from the Tamil ethnic group, slitting their throats and dumping their bodies into a sewer in 2000. Three of the victims were children.The pardon brought outrage from rights activists and opposition politicians, but little obvious reaction from the broader Sri Lankan public, which is under a strict curfew in order to contain the spread of the coronavirus.President Gotabaya Rajapaksa instructed the Ministry of Justice to release Mr. Ratnayake from prison on Thursday. Mr. Rajapaksa, who was elected in November, is himself accused of having ordered war crimes during the civil war, when he served as defense secretary.The Sri Lankan government did not immediately respond to requests for comment.The United Nations called the pardon an “affront to victims” and another example of Sri Lanka’s failure to hold war criminals to account.\nLatest Updates: Global Coronavirus Outbreak\nThree top U.S. health officials will quarantine after White House staff members tested positive.\nA new cluster in China highlights the challenge of controlling infections as countries open up.\nThe U.F.C. made its return, the first major U.S. sporting event to take place in weeks.\nSee more updates\nUpdated 19m ago\nMore live coverage:\nMarkets\nU.S.\nNew York\n“This was one of the rare human rights cases from the decades-long conflict that had ever reached conviction,” a spokesman for the U.N. High Commissioner for Human Rights, Michele Blanchett, said on Friday.The chairman of the House Foreign Affairs Committee in the United States, Representative Eliot Engel, condemned the pardon in a Twitter post on Friday.Amnesty International accused the government of taking advantage of a world distracted by the coronavirus to dodge international condemnation.“Using the pandemic as an opportunity to release those convicted of heinous crimes is reprehensible,” Amnesty International said in a statement.The eight Tamil civilians were killed as they returned to their homes to salvage belongings"
1,EU forecasts 'recession of historic proportions' this year,"year” due to the impact of the coronavirusBy LORNE COOK Associated PressMay 6, 2020, 1:32 PM4 min read4 min readShare to FacebookShare to TwitterEmail this article6:46News headlines today: May 8, 2020Catch up on the developing stories making headlines.The Associated PressBRUSSELS --\nThe European Union predicted Wednesday “a recession of historic proportions this year” due to the impact of the coronavirus as it released its first official estimates of the damage the pandemic is inflicting on the bloc’s economy.\nThe 27-nation EU economy is predicted to contract by 7.5% this year, before growing about 6% in 2021, assuming countries steadily ease their lockdowns.\nThe group of 19 nations using the euro as their currency will see a record decline of 7.75% this year, and grow by 6.25% in 2021, the European Commission said in its\nSpring economic forecast.\n“It is now quite clear that the EU has entered the deepest economic recession in its history,"" EU Economy Commissioner Paolo Gentiloni told reporters in Brussels. As the virus hit, “economic activity in the EU dropped by around one third practically overnight,"

In [5]:
# GENSIM 
def tag_corp(corp,tag):
    return gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(corp), ([tag]))


tagged_document = [tag_corp(x,y) for x,y in list(zip(all_news['headlines'],all_news['content']))]
model = gensim.models.doc2vec.Doc2Vec(vector_size=300, min_count=10, epochs=50)
vocab = model.build_vocab(tagged_document)
model.train(tagged_document, total_examples=model.corpus_count, epochs=model.epochs)


2020-05-21 17:16:06,342 : INFO : collecting all words and their counts
2020-05-21 17:16:06,342 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2020-05-21 17:16:06,535 : INFO : PROGRESS: at example #10000, processed 132972 words (693386/s), 20823 word types, 9816 tags
2020-05-21 17:16:06,695 : INFO : collected 27922 word types and 16934 unique tags from a corpus of 17252 examples and 226334 words
2020-05-21 17:16:06,699 : INFO : Loading a fresh vocabulary
2020-05-21 17:16:06,730 : INFO : effective_min_count=10 retains 2837 unique words (10% of original 27922, drops 25085)
2020-05-21 17:16:06,731 : INFO : effective_min_count=10 leaves 180170 word corpus (79% of original 226334, drops 46164)
2020-05-21 17:16:06,749 : INFO : deleting the raw counts dictionary of 27922 items
2020-05-21 17:16:06,749 : INFO : sample=0.001 downsamples 38 most-common words
2020-05-21 17:16:06,753 : INFO : downsampling leaves estimated 138254 word corpus (76.7% of prior 180170)
20

2020-05-21 17:16:52,460 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-21 17:16:52,467 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-21 17:16:52,467 : INFO : EPOCH - 10 : training on 226334 raw words (155437 effective words) took 3.2s, 48211 effective words/s
2020-05-21 17:16:53,668 : INFO : EPOCH 11 - PROGRESS: at 30.92% examples, 40684 words/s, in_qsize 5, out_qsize 0
2020-05-21 17:16:54,770 : INFO : EPOCH 11 - PROGRESS: at 61.19% examples, 41928 words/s, in_qsize 5, out_qsize 0
2020-05-21 17:16:55,687 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-05-21 17:16:55,786 : INFO : EPOCH 11 - PROGRESS: at 96.95% examples, 45871 words/s, in_qsize 1, out_qsize 1
2020-05-21 17:16:55,787 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-21 17:16:55,812 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-21 17:16:55,814 : INFO : EPOCH - 11 : training on 226334 raw wo

2020-05-21 17:17:32,577 : INFO : EPOCH 23 - PROGRESS: at 70.10% examples, 45739 words/s, in_qsize 5, out_qsize 0
2020-05-21 17:17:33,127 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-05-21 17:17:33,348 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-21 17:17:33,372 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-21 17:17:33,380 : INFO : EPOCH - 23 : training on 226334 raw words (155515 effective words) took 3.2s, 48756 effective words/s
2020-05-21 17:17:34,395 : INFO : EPOCH 24 - PROGRESS: at 17.51% examples, 27079 words/s, in_qsize 5, out_qsize 0
2020-05-21 17:17:35,418 : INFO : EPOCH 24 - PROGRESS: at 47.80% examples, 37022 words/s, in_qsize 5, out_qsize 0
2020-05-21 17:17:36,473 : INFO : EPOCH 24 - PROGRESS: at 74.66% examples, 37762 words/s, in_qsize 5, out_qsize 0
2020-05-21 17:17:36,972 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-05-21 17:17:37,102 : INFO : worker thread fi

2020-05-21 17:18:11,213 : INFO : EPOCH - 35 : training on 226334 raw words (155784 effective words) took 3.0s, 51919 effective words/s
2020-05-21 17:18:12,403 : INFO : EPOCH 36 - PROGRESS: at 30.92% examples, 41028 words/s, in_qsize 5, out_qsize 0
2020-05-21 17:18:13,516 : INFO : EPOCH 36 - PROGRESS: at 70.10% examples, 47643 words/s, in_qsize 5, out_qsize 0
2020-05-21 17:18:14,127 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-05-21 17:18:14,232 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-21 17:18:14,244 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-21 17:18:14,246 : INFO : EPOCH - 36 : training on 226334 raw words (155412 effective words) took 3.0s, 51561 effective words/s
2020-05-21 17:18:15,414 : INFO : EPOCH 37 - PROGRESS: at 30.44% examples, 41251 words/s, in_qsize 5, out_qsize 0
2020-05-21 17:18:16,636 : INFO : EPOCH 37 - PROGRESS: at 70.10% examples, 46081 words/s, in_qsize 5, out_qsize 0
2020-

2020-05-21 17:18:52,357 : INFO : EPOCH 48 - PROGRESS: at 92.67% examples, 34728 words/s, in_qsize 2, out_qsize 1
2020-05-21 17:18:52,368 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-05-21 17:18:52,472 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-21 17:18:52,503 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-21 17:18:52,505 : INFO : EPOCH - 48 : training on 226334 raw words (155456 effective words) took 4.3s, 36193 effective words/s
2020-05-21 17:18:53,546 : INFO : EPOCH 49 - PROGRESS: at 22.04% examples, 33712 words/s, in_qsize 6, out_qsize 0
2020-05-21 17:18:54,599 : INFO : EPOCH 49 - PROGRESS: at 52.28% examples, 39499 words/s, in_qsize 5, out_qsize 0
2020-05-21 17:18:55,625 : INFO : EPOCH 49 - PROGRESS: at 74.66% examples, 37560 words/s, in_qsize 5, out_qsize 0
2020-05-21 17:18:56,226 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-05-21 17:18:56,308 : INFO : worker thread fi

In [19]:
# print(cnn_model.wv.similar_by_vector('cnn', topn=10, restrict_vocab=None))
# print(cnn_model.wv.similar_by_vector('china', topn=10, restrict_vocab=None))
print(model.wv.similar_by_vector('vaccine', topn=5, restrict_vocab=None))
print(model.wv.similar_by_vector('lab', topn=5, restrict_vocab=None))
print(model.wv.similar_by_vector('theory', topn=5, restrict_vocab=None))
print(model.wv.similar_by_vector('masks', topn=5, restrict_vocab=None))



[('trials', 0.6125087738037109),
 ('treatments', 0.5940625071525574),
 ('antibodies', 0.5329806208610535),
 ('treatment', 0.5315247774124146),
 ('research', 0.5265873670578003)]
[('wuhan', 0.5987124443054199),
 ('chinese', 0.5659451484680176),
 ('theory', 0.5418941378593445),
 ('epicenter', 0.5316033363342285),
 ('uss', 0.5132399797439575)]
[('conspiracy', 0.7619000673294067),
 ('theories', 0.758734941482544),
 ('dismisses', 0.6047114133834839),
 ('disinfectant', 0.5992051362991333),
 ('intel', 0.5737899541854858)]
[('mask', 0.6599047183990479),
 ('protective', 0.5635757446289062),
 ('wear', 0.5526955127716064),
 ('wearing', 0.5453240871429443),
 ('gloves', 0.519646167755127)]


In [7]:
from pprint import pprint as print
from gensim.summarization import summarize
model.init_sims(replace=True)
from scipy import spatial

from scipy import spatial


list_content = all_news['content'].values.tolist()[112:190]
list_headlines = all_news['headlines'].values.tolist()[112:190]

similarities=[]

for s,h in zip(list_content,list_headlines):

    summary = (summarize(s, word_count=30))
    
    similarities.append(h)
    vec1 = model.infer_vector(s.split())
    vec2 = model.infer_vector(h.split())
    similarities.append(summary)
    similarity = spatial.distance.cosine(vec1, vec2)
    
    similarities.append(similarity)


   
    


2020-05-21 17:18:59,097 : INFO : precomputing L2-norms of doc weight vectors
2020-05-21 17:18:59,139 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-05-21 17:18:59,145 : INFO : built Dictionary(149 unique tokens: ['australia', 'edit', 'job', 'search', 'date']...) from 61 documents (total 255 corpus positions)
2020-05-21 17:18:59,146 : INFO : Building graph
2020-05-21 17:18:59,148 : INFO : Filling graph
2020-05-21 17:18:59,155 : INFO : Removing unreachable nodes of graph
2020-05-21 17:18:59,156 : INFO : Pagerank graph
2020-05-21 17:18:59,164 : INFO : Sorting pagerank scores
2020-05-21 17:18:59,239 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-05-21 17:18:59,243 : INFO : built Dictionary(191 unique tokens: ['australia', 'edit', 'job', 'search', 'date']...) from 61 documents (total 263 corpus positions)
2020-05-21 17:18:59,247 : INFO : Building graph
2020-05-21 17:18:59,249 : INFO : Filling graph
2020-05-21 17:18:59,255 : INFO : Removing unreachab

2020-05-21 17:19:00,342 : INFO : Removing unreachable nodes of graph
2020-05-21 17:19:00,344 : INFO : Pagerank graph
2020-05-21 17:19:00,349 : INFO : Sorting pagerank scores
2020-05-21 17:19:00,404 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-05-21 17:19:00,407 : INFO : built Dictionary(137 unique tokens: ['act', 'assist', 'contract', 'england', 'evict']...) from 12 documents (total 208 corpus positions)
2020-05-21 17:19:00,408 : INFO : Building graph
2020-05-21 17:19:00,409 : INFO : Filling graph
2020-05-21 17:19:00,411 : INFO : Removing unreachable nodes of graph
2020-05-21 17:19:00,412 : INFO : Pagerank graph
2020-05-21 17:19:00,416 : INFO : Sorting pagerank scores
2020-05-21 17:19:00,475 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-05-21 17:19:00,479 : INFO : built Dictionary(165 unique tokens: ['australia', 'edit', 'job', 'search', 'date']...) from 62 documents (total 277 corpus positions)
2020-05-21 17:19:00,481 : INFO : Building grap

2020-05-21 17:19:01,608 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-05-21 17:19:01,612 : INFO : built Dictionary(200 unique tokens: ['australia', 'edit', 'job', 'search', 'date']...) from 57 documents (total 269 corpus positions)
2020-05-21 17:19:01,613 : INFO : Building graph
2020-05-21 17:19:01,615 : INFO : Filling graph
2020-05-21 17:19:01,620 : INFO : Removing unreachable nodes of graph
2020-05-21 17:19:01,620 : INFO : Pagerank graph
2020-05-21 17:19:01,629 : INFO : Sorting pagerank scores
2020-05-21 17:19:01,679 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-05-21 17:19:01,685 : INFO : built Dictionary(132 unique tokens: ['australia', 'edit', 'job', 'search', 'date']...) from 98 documents (total 202 corpus positions)
2020-05-21 17:19:01,687 : INFO : Building graph
2020-05-21 17:19:01,688 : INFO : Filling graph
2020-05-21 17:19:01,693 : INFO : Removing unreachable nodes of graph
2020-05-21 17:19:01,697 : INFO : Pagerank graph
2020-05-21 

2020-05-21 17:19:02,699 : INFO : built Dictionary(177 unique tokens: ['australia', 'edit', 'job', 'search', 'date']...) from 62 documents (total 255 corpus positions)
2020-05-21 17:19:02,700 : INFO : Building graph
2020-05-21 17:19:02,701 : INFO : Filling graph
2020-05-21 17:19:02,707 : INFO : Removing unreachable nodes of graph
2020-05-21 17:19:02,708 : INFO : Pagerank graph
2020-05-21 17:19:02,713 : INFO : Sorting pagerank scores
2020-05-21 17:19:02,770 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-05-21 17:19:02,775 : INFO : built Dictionary(167 unique tokens: ['killer', 'serial', 'unknown', 'victim', 'coronaviru']...) from 61 documents (total 266 corpus positions)
2020-05-21 17:19:02,777 : INFO : Building graph
2020-05-21 17:19:02,779 : INFO : Filling graph
2020-05-21 17:19:02,786 : INFO : Removing unreachable nodes of graph
2020-05-21 17:19:02,786 : INFO : Pagerank graph
2020-05-21 17:19:02,797 : INFO : Sorting pagerank scores
2020-05-21 17:19:02,856 : INFO :

2020-05-21 17:19:03,867 : INFO : Removing unreachable nodes of graph
2020-05-21 17:19:03,872 : INFO : Pagerank graph
2020-05-21 17:19:03,883 : INFO : Sorting pagerank scores
2020-05-21 17:19:03,950 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-05-21 17:19:03,955 : INFO : built Dictionary(185 unique tokens: ['australia', 'edit', 'job', 'search', 'date']...) from 67 documents (total 251 corpus positions)
2020-05-21 17:19:03,956 : INFO : Building graph
2020-05-21 17:19:03,956 : INFO : Filling graph
2020-05-21 17:19:03,962 : INFO : Removing unreachable nodes of graph
2020-05-21 17:19:03,962 : INFO : Pagerank graph
2020-05-21 17:19:03,971 : INFO : Sorting pagerank scores
2020-05-21 17:19:04,039 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-05-21 17:19:04,046 : INFO : built Dictionary(194 unique tokens: ['killer', 'serial', 'coronaviru', 'cb', 'coverag']...) from 54 documents (total 299 corpus positions)
2020-05-21 17:19:04,047 : INFO : Building gr

2020-05-21 17:19:05,045 : INFO : Filling graph
2020-05-21 17:19:05,054 : INFO : Removing unreachable nodes of graph
2020-05-21 17:19:05,054 : INFO : Pagerank graph
2020-05-21 17:19:05,065 : INFO : Sorting pagerank scores
2020-05-21 17:19:05,141 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-05-21 17:19:05,141 : INFO : built Dictionary(181 unique tokens: ['australia', 'edit', 'job', 'search', 'date']...) from 55 documents (total 263 corpus positions)
2020-05-21 17:19:05,145 : INFO : Building graph
2020-05-21 17:19:05,148 : INFO : Filling graph
2020-05-21 17:19:05,157 : INFO : Removing unreachable nodes of graph
2020-05-21 17:19:05,158 : INFO : Pagerank graph
2020-05-21 17:19:05,168 : INFO : Sorting pagerank scores
2020-05-21 17:19:05,224 : WARNING : Input text is expected to have at least 10 sentences.
2020-05-21 17:19:05,226 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-05-21 17:19:05,228 : INFO : built Dictionary(170 unique tokens: ['alarm', 

In [8]:
similarities

['Fifth of primary children afraid to leave house over Covid-19, survey finds | World news | The Guardian',
 'As many as one in five primary age children are afraid to leave their homes and are worried there will not be enough food to eat during the course of the Covid-19 outbreak, according to the findings of a survey.',
 0.6494123041629791,
 'Coronavirus puts vulnerable UK children at greater risk, campaigners warn | Society | The Guardian',
 'Coronavirus puts vulnerable UK children at greater risk, campaigners warn\nShare on Facebook\nShare on Twitter',
 0.7331650853157043,
 "'Stranded at sea': cruise ships around the world are adrift as ports turn them away | World news | The Guardian",
 'The Zaandam is one of at least 10 ships around the world – carrying nearly 10,000 passengers – still stuck at sea after having been turned away from their destination ports in the face of the Covid-19 pandemic, according to a Guardian analysis.',
 0.6133175194263458,
 'Walmart hit with wrongful de